<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/EEG_stream_connectivity_Generate_Drums_with_WaveGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!ls ~/.jupyter/
#!echo c.NotebookApp.iopub_data_rate_limit = 10000000 >> ~/.jupyter/jupyter_notebook_config.py
#!cat ~/.jupyter/jupyter_notebook_config.py

In [3]:
  duration=5*1/8
  #duration=2*1/8
  #duration=1*1/8
  overlap=duration-0.1
  xsize=300
  ysize=300
  hz=44100

  dim = 100
  #dim = 512


In [4]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10

In [5]:
##biosemi16-2:
ch_names = ['FP1','F3','T7','C3','P3','Pz','O1','O2','P4','C4','T8','F4','FP2','Fz']
ch_locations=[0,3,6,7,11,12,14,16,18,22,23,26,29,30]

##biosemi16:
#ch_names = ['FP1','F3','T7','C3','P3','Pz','O1','Oz','O2','P4','C4','T8','F4','FP2','Fz','Cz']
#ch_locations=[0,3,6,7,11,12,14,15,16,18,22,23,26,29,30,31]
##biosemi32:
#ch_names = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','FP2','Fz','Cz']
#ch_locations=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
##Bernard's 19ch:
#ch_names = ["FP2","FP1","O2","T6","T4","F8","F4","C4","P4","F3","C3","P3","O1","T5","T3","F7","FZ","PZ"]#,"other"]
#ch_locations=[4,24,0,1,2,3,5,6,7,25,26,27,28,29,30,31,16,12]#,8]
##Bernard's 2ch:
#ch_names = ["FP2","FP1"]#,"other"]
#ch_locations=[4,24]#,8]

#ch_names = ['O2','T6','T4','F8','FP2','F4','C4','P4','other','','','','PZ','','','','FZ','','','','','','','','FP1','F3','C3','P3','O1','T5','T3','F7']


In [54]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''

/**/

//Joshua Brewster, GPL (copyleft)

//import 'regenerator-runtime/runtime' //For async functions on node\\

 class eeg32 { //Contains structs and necessary functions/API calls to analyze serial data for the FreeEEG32

    constructor(
		onDecodedCallback = this.onDecodedCallback,
		onConnectedCallback = this.onConnectedCallback,
		onDisconnectedCallback = this.onDisconnectedCallback,
		CustomDecoder = this.decode,
		baudrate = 921600//115200
		) {

		this.onDecodedCallback = onDecodedCallback;
		this.onConnectedCallback = onConnectedCallback;
		this.onDisconnectedCallback = onDisconnectedCallback;
		this.decode = CustomDecoder;
		//Free EEG 32 data structure:
        
        //    [stop byte, start byte, counter byte, 32x3 channel data bytes (24 bit), 3x2 accelerometer data bytes, stop byte, start byte...] Gyroscope not enabled yet but would be printed after the accelerometer..
        //    Total = 105 bytes/line
        
		this.connected = false;
		this.subscribed = false;
        this.buffer = [];
        this.startByte = 160; // Start byte value
		this.stopByte = 192; // Stop byte value
		this.searchString = new Uint8Array([this.stopByte,this.startByte]); //Byte search string
		this.readRate = 16.666667; //Throttle EEG read speed. (1.953ms/sample min @103 bytes/line)
		this.readBufferSize = 2000; //Serial read buffer size, increase for slower read speeds (~1030bytes every 20ms) to keep up with the stream (or it will crash)

		this.sps = 512; // Sample rate
		this.nChannels = 32;
		this.nPeripheralChannels = 6; // accelerometer and gyroscope (2 bytes * 3 coordinates each)
		this.updateMs = 1000/this.sps; //even spacing
		this.stepSize = 1/Math.pow(2,24);
		this.vref = 2.50; //2.5V voltage ref +/- 250nV
		this.gain = 8;

		this.vscale = (this.vref/this.gain)*this.stepSize; //volts per step.
		this.uVperStep = 1000000 * ((this.vref/this.gain)*this.stepSize); //uV per step.
		this.scalar = 1/(1000000 / ((this.vref/this.gain)*this.stepSize)); //steps per uV.

		this.maxBufferedSamples = this.sps*60*2; //max samples in buffer this.sps*60*nMinutes = max minutes of data
		
		this.data = { //Data object to keep our head from exploding. Get current data with e.g. this.data.A0[this.data.count-1]
			count: 0,
			startms: 0,
			ms: [],
			'A0': [],'A1': [],'A2': [],'A3': [],'A4': [],'A5': [],'A6': [],'A7': [], //ADC 0
			'A8': [],'A9': [],'A10': [],'A11': [],'A12': [],'A13': [],'A14': [],'A15': [], //ADC 1
			'A16': [],'A17': [],'A18': [],'A19': [],'A20': [],'A21': [],'A22': [],'A23': [], //ADC 2
			'A24': [],'A25': [],'A26': [],'A27': [],'A28': [],'A29': [],'A30': [],'A31': [], //ADC 3
			'Ax': [], 'Ay': [], 'Az': [], 'Gx': [], 'Gy': [], 'Gz': []  //Peripheral data (accelerometer, gyroscope)
		};

    //this.lastnewLines = 0;
    this.bufferednewLines = 0;
    this.data_slice=[];
    this.data_slice_size=512*(5*1/8+0.1);
    //this.data_slice_size=512*(2*1/8+0.1);
    //this.data_slice_size=512*(1*1/8+0.1);
    this.ready_to_send_data = false;


		this.resetDataBuffers();

		//navigator.serial utils
		if(!navigator.serial){
			console.error("`navigator.serial not found! Enable #enable-experimental-web-platform-features in chrome://flags (search 'experimental')")
		}
		this.port = null;
		this.reader = null;
		this.baudrate = baudrate;

	}
	
	resetDataBuffers(){
		this.data.count = 0;
		this.data.startms = 0;
		for(const prop in this.data) {
			if(typeof this.data[prop] === "object"){
				this.data[prop] = new Array(this.maxBufferedSamples).fill(0);
			}
		}
	}

	setScalar(gain=24,stepSize=1/(Math.pow(2,23)-1),vref=4.50) {
        this.stepSize = stepSize;
		this.vref = vref; //2.5V voltage ref +/- 250nV
		this.gain = gain;

		this.vscale = (this.vref/this.gain)*this.stepSize; //volts per step.
		this.uVperStep = 1000000 * ((this.vref/this.gain)*this.stepSize); //uV per step.
		this.scalar = 1/(1000000 / ((this.vref/this.gain)*this.stepSize)); //steps per uV.
    }

	getLatestData(channel="A0",count=1) { //Return slice of specified size of the latest data from the specified channel
		let ct = count;
		if(ct <= 1) {
			return [this.data[channel][this.data.count-1]];
		}
		else {
			if(ct > this.data.count) {
				ct = this.data.count;
			}
			return this.data[channel].slice(this.data.count-ct,this.data.count);
		}
	}

    bytesToInt16(x0,x1){
		return x0 * 256 + x1;
    }

    int16ToBytes(y){ //Turns a 24 bit int into a 3 byte sequence
        return [y & 0xFF , (y >> 8) & 0xFF];
    }

    bytesToInt24(x0,x1,x2){ //Turns a 3 byte sequence into a 24 bit int
        return x0 * 65536 + x1 * 256 + x2;
    }

    int24ToBytes(y){ //Turns a 24 bit int into a 3 byte sequence
        return [y & 0xFF , (y >> 8) & 0xFF , (y >> 16) & 0xFF];
    }

    decode(buffer = this.buffer) { //returns true if successful, returns false if not

		var needle = this.searchString
		var haystack = buffer;
		var search = this.boyerMoore(needle);
		var skip = search.byteLength;
		var indices = [];
		let newLines = 0;
    
		for (var i = search(haystack); i !== -1; i = search(haystack, i + skip)) {
			indices.push(i);
		}
		//console.log(indices);
		if(indices.length >= 2){
			for(let k = 1; k < indices.length; k++) {
				if(indices[k] - indices[k-1] !== 105) {
					
				} //This is not a valid sequence going by size, drop sequence and return
				else {
					var line = buffer.slice(indices[k-1],indices[k]+1); //Splice out this line to be decoded
					
					// line[0] = stop byte, line[1] = start byte, line[2] = counter, line[3:99] = ADC data 32x3 bytes, line[100-104] = Accelerometer data 3x2 bytes

					//line found, decode.
					if(this.data.count < this.maxBufferedSamples){
						this.data.count++;
					}

					if(this.data.count-1 === 0) {this.data.ms[this.data.count-1]= Date.now(); this.data.startms = this.data.ms[0];}
					else {
						this.data.ms[this.data.count-1]=this.data.ms[this.data.count-2]+this.updateMs;
						
						if(this.data.count >= this.maxBufferedSamples) {
							this.data.ms.splice(0,5120);
							this.data.ms.push(new Array(5120).fill(0));
						}
					}//Assume no dropped samples
				
					for(var i = 3; i < 99; i+=3) {
						var channel = "A"+(i-3)/3;
						this.data[channel][this.data.count-1]=this.bytesToInt24(line[i],line[i+1],line[i+2]);
						if(this.data.count >= this.maxBufferedSamples) { 
							this.data[channel].splice(0,5120);
							this.data[channel].push(new Array(5120).fill(0));//shave off the last 10 seconds of data if buffer full (don't use shift())
						}
							//console.log(this.data[channel][this.data.count-1],indices[k], channel)
					}

					this.data["Ax"][this.data.count-1]=this.bytesToInt16(line[99],line[100]);
					this.data["Ay"][this.data.count-1]=this.bytesToInt16(line[101],line[102]);
					this.data["Az"][this.data.count-1]=this.bytesToInt16(line[103],line[104]);

					
					if(this.data.count >= this.maxBufferedSamples) { 
						this.data["Ax"].splice(0,5120);
						this.data["Ay"].splice(0,5120);
						this.data["Az"].splice(0,5120);
						this.data["Ax"].push(new Array(5120).fill(0))
						this.data["Ay"].push(new Array(5120).fill(0))
						this.data["Az"].push(new Array(5120).fill(0))
						this.data.count -= 5120;
					}
					//console.log(this.data)
					newLines++;
					//console.log(indices[k-1],indices[k])
					//console.log(buffer[indices[k-1],buffer[indices[k]]])
					//indices.shift();
				}
				
			}
			if(newLines > 0) buffer.splice(0,indices[indices.length-1]);
   
			return newLines;
			//Continue
		}
		//else {this.buffer = []; return false;}
	}

	//Callbacks
	onDecodedCallback(newLinesInt){
		///console.log("new samples:", newLinesInt);
    //>add slicing
    this.bufferednewLines=this.bufferednewLines+newLinesInt;
    if(this.bufferednewLines>this.data_slice_size)
      {
        this.bufferednewLines=this.data_slice_size;
        //this.takeSlice(this.data.count-1-this.bufferednewLines,this.data.count-1);
        //this.takeSliceandsend(this.data.count-1-this.bufferednewLines,this.data.count-1);
        //this.takeSliceandsend3(this.data.count-1-newLinesInt,this.data.count-1);
        //this.takeSliceandsend2(this.data.count-1-newLinesInt,this.data.count-1);
        //this.bufferednewLines=0;
      }
        this.takeSlice(this.data.count-1-this.bufferednewLines,this.data.count-1);
        //this.takeSliceandsend(this.data.count-1-newLinesInt,this.data.count-1);
    //add slicing<
	}

	onConnectedCallback() {
		console.log("port connected!");
	}

	onDisconnectedCallback() {
		console.log("port disconnected!");
	}

	onReceive(value){
		this.buffer.push(...value);

		let newLines = this.decode(this.buffer);
		//console.log(this.data)
		//console.log("decoding... ", this.buffer.length)
		if(newLines !== false && newLines !== 0 && !isNaN(newLines) ) this.onDecodedCallback(newLines);
	}

	async onPortSelected(port,baud=this.baudrate) {
		try{
			try {
				await port.open({ baudRate: baud, bufferSize: this.readBufferSize });
				this.onConnectedCallback();
				this.connected = true;
				this.subscribed = true;
				this.subscribe(port);//this.subscribeSafe(port);
		
			} //API inconsistency in syntax between linux and windows
			catch {
				await port.open({ baudrate: baud, buffersize: this.readBufferSize });
				this.onConnectedCallback();
				this.connected = true;
				this.subscribed = true;
				this.subscribe(port);//this.subscribeSafe(port);
			}
		}
		catch(err){
			console.log(err);
			this.connected = false;
		}
	}

	async subscribe(port){
		if (this.port.readable && this.subscribed === true) {
			this.reader = port.readable.getReader();
			const streamData = async () => {
				try {
					const { value, done } = await this.reader.read();
					if (done || this.subscribed === false) {
						// Allow the serial port to be closed later.
						await this.reader.releaseLock();
						
					}
					if (value) {
						//console.log(value.length);
						try{
							this.onReceive(value);
						}
						catch (err) {console.log(err)}
						//console.log("new Read");
						//console.log(this.decoder.decode(value));
					}
					if(this.subscribed === true) {
						setTimeout(()=>{streamData();}, this.readRate);//Throttled read 1/512sps = 1.953ms/sample @ 103 bytes / line or 1030bytes every 20ms
					}
				} catch (error) {
					console.log(error);// TODO: Handle non-fatal read error.
                    if(error.message.includes('framing') || error.message.includes('overflow') || error.message.includes('Overflow') || error.message.includes('break')) {
                        this.subscribed = false;
                        setTimeout(async ()=>{
							try{
                            if (this.reader) {
                                await this.reader.releaseLock();
                                this.reader = null;
                            }
							} catch (er){ console.error(er);}
                            this.subscribed = true; 
                            this.subscribe(port);
                            //if that fails then close port and reopen it
                        },30); //try to resubscribe 
                    } else if (error.message.includes('parity') || error.message.includes('Parity') || error.message.includes('overrun') ) {
                        if(this.port){
                            this.subscribed = false;
                            setTimeout(async () => {
								try{
                                if (this.reader) {
                                    await this.reader.releaseLock();
                                    this.reader = null;
                                }
                                await port.close();
								} catch (er){ console.error(er);}
                                //this.port = null;
                                this.connected = false;
                                setTimeout(()=>{this.onPortSelected(this.port)},100); //close the port and reopen
                            }, 50);
                        }
                    }
                     else {
                        this.closePort();	
                    }	
				}
			}
			streamData();
		}
	}

	//Unfinished
	async subscribeSafe(port) { //Using promises instead of async/await to cure hangs when the serial update does not meet tick requirements
		var readable = new Promise((resolve,reject) => {
			while(this.port.readable && this.subscribed === true){
				this.reader = port.readable.getReader();
				var looper = true;
				var prom1 = new Promise((resolve,reject) => {
					return this.reader.read();
				});

				var prom2 = new Promise((resolve,reject) => {
					setTimeout(resolve,100,"readfail");
				});
				while(looper === true ) {
					//console.log("reading...");
					Promise.race([prom1,prom2]).then((result) => {
						console.log("newpromise")
						if(result === "readfail"){
							console.log(result);
						}
						else{
							const {value, done} = result;
							if(done === true || this.subscribed === true) { var donezo = new Promise((resolve,reject) => {
								resolve(this.reader.releaseLock())}).then(() => {
									looper = false;
									return;
								});
							}
							else{
								this.onReceive(value);
							}
						}
					});
				}
			}
			resolve("not readable");
		});
	}

	async closePort(port=this.port) {
		//if(this.reader) {this.reader.releaseLock();}
		if(this.port){
			this.subscribed = false;
			setTimeout(async () => {
				if (this.reader) {
					await this.reader.releaseLock();
					this.reader = null;
				}
				await port.close();
				this.port = null;
				this.connected = false;
				this.onDisconnectedCallback();
			}, 100);
		}
	}

	async setupSerialAsync(baudrate=this.baudrate) { //You can specify baudrate just in case

		const filters = [
			{ usbVendorId: 0x10c4, usbProductId: 0x0043 } //CP2102 filter (e.g. for UART via ESP32)
		];

		this.port = await navigator.serial.requestPort();
		navigator.serial.addEventListener("disconnect",(e) => {
			this.closePort(this.port);
		});
		this.onPortSelected(this.port,baudrate);

		//navigator.serial.addEventListener("onReceive", (e) => {console.log(e)});//this.onReceive(e));

	}


	//Boyer Moore fast byte search method copied from https://codereview.stackexchange.com/questions/20136/uint8array-indexof-method-that-allows-to-search-for-byte-sequences
	asUint8Array(input) {
		if (input instanceof Uint8Array) {
			return input;
		} else if (typeof(input) === 'string') {
			// This naive transform only supports ASCII patterns. UTF-8 support
			// not necessary for the intended use case here.
			var arr = new Uint8Array(input.length);
			for (var i = 0; i < input.length; i++) {
			var c = input.charCodeAt(i);
			if (c > 127) {
				throw new TypeError("Only ASCII patterns are supported");
			}
			arr[i] = c;
			}
			return arr;
		} else {
			// Assume that it's already something that can be coerced.
			return new Uint8Array(input);
		}
	}

	boyerMoore(patternBuffer) {
		// Implementation of Boyer-Moore substring search ported from page 772 of
		// Algorithms Fourth Edition (Sedgewick, Wayne)
		// http://algs4.cs.princeton.edu/53substring/BoyerMoore.java.html
		
//		USAGE:
			// needle should be ASCII string, ArrayBuffer, or Uint8Array
			// haystack should be an ArrayBuffer or Uint8Array
//			var search = boyerMoore(needle);
//			var skip = search.byteLength;
//			var indices = [];
//			for (var i = search(haystack); i !== -1; i = search(haystack, i + skip)) {
//				indices.push(i);
//			}
		
		var pattern = this.asUint8Array(patternBuffer);
		var M = pattern.length;
		if (M === 0) {
			throw new TypeError("patternBuffer must be at least 1 byte long");
		}
		// radix
		var R = 256;
		var rightmost_positions = new Int32Array(R);
		// position of the rightmost occurrence of the byte c in the pattern
		for (var c = 0; c < R; c++) {
			// -1 for bytes not in pattern
			rightmost_positions[c] = -1;
		}
		for (var j = 0; j < M; j++) {
			// rightmost position for bytes in pattern
			rightmost_positions[pattern[j]] = j;
		}
		var boyerMooreSearch = (txtBuffer, start, end) => {
			// Return offset of first match, -1 if no match.
			var txt = this.asUint8Array(txtBuffer);
			if (start === undefined) start = 0;
			if (end === undefined) end = txt.length;
			var pat = pattern;
			var right = rightmost_positions;
			var lastIndex = end - pat.length;
			var lastPatIndex = pat.length - 1;
			var skip;
			for (var i = start; i <= lastIndex; i += skip) {
				skip = 0;
				for (var j = lastPatIndex; j >= 0; j--) {
				var c = txt[i + j];
				if (pat[j] !== c) {
					skip = Math.max(1, j - right[c]);
					break;
				}
				}
				if (skip === 0) {
				return i;
				}
			}
			return -1;
		};
		boyerMooreSearch.byteLength = pattern.byteLength;
		return boyerMooreSearch;
	}
	//---------------------end copy/pasted solution------------------------

    async  takeSlice(data_slice_from,data_slice_to) {
      //var data_slice_from=this.data.count-1-this.bufferednewLines;
      //var data_slice_to=this.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
        //this.lastnewLines=this.bufferednewLines;
    }
    async  takeSliceandsend(data_slice_from,data_slice_to) {
      //var data_slice_from=this.data.count-1-this.bufferednewLines;
      //var data_slice_to=this.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
        //this.lastnewLines=this.bufferednewLines;

    }
        async  takeSliceandsend2(data_slice_from,data_slice_to) {
      //var data_slice_from=this.data.count-1-this.bufferednewLines;
      //var data_slice_to=this.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
        //this.lastnewLines=this.bufferednewLines;
      this.send3();
    }
async send3(){
 //document.body.appendChild(document.createTextNode(`asJSON is of type ${typeof this.data_slice}`));
 //var array_to_send_as_json1 = JSON.stringify(this.data_slice);
  //document.body.appendChild(document.createTextNode(array_to_send_as_json1));
 if(this.ready_to_send_data&&this.data_slice.length){
      this.ready_to_send_data = false;
  // (async () => {
  //const buffer = new Uint8Array(1);
  //for (let i = 0; i < buffer.byteLength; ++i) {
  //  buffer[i] = i
  //}
  //document.body.appendChild(document.createTextNode(JSON.stringify([buffer])));
  //document.body.appendChild(document.createTextNode(JSON.stringify([buffer.buffer.byteLength])));
  //var data_slice_out = this.data_slice;
  //var array_to_send = this.data_slice;
  //document.body.appendChild(document.createTextNode(`asJSON is of type ${typeof array_to_send}`));
  //var array_to_send_as_json = JSON.stringify(array_to_send);
  
  var array_to_send_as_json = JSON.stringify(this.data_slice);
  
  //document.body.appendChild(document.createTextNode(array_to_send_as_json));
  document.body.appendChild(document.createTextNode('sending ready'));
  const channel = await google.colab.kernel.comms.open('comm_target', array_to_send_as_json, []);
  //const channel = await google.colab.kernel.comms.open('comm_target', 'the data', [buffer.buffer]);
  this.data_slice = [];
  document.body.appendChild(document.createTextNode('sending'));

  let success = false;
  for await (const message of channel.messages) {
    if (message.data.response == 'got comm open!') {
      document.body.appendChild(document.createTextNode('ready'));
      this.ready_to_send_data = true;
      //document.body.appendChild(document.createTextNode(JSON.stringify([message])));
      //const responseBuffer = new Uint8Array(message.buffers[0]);
      //for (let i = 0; i < buffer.length; ++i) {
      //  if (responseBuffer[i] != buffer[i]) {
      //    console.error('comm buffer different at ' + i);
      //    return;
      //  }
      //}
      // Close the channel once the expected message is received. This should
      // cause the messages iterator to complete and for the for-await loop to
      // end.
      channel.close();
    }
  }
 }
}    
        async  takeSliceandsend3(data_slice_from,data_slice_to) {
      //var data_slice_from=this.data.count-1-this.bufferednewLines;
      //var data_slice_to=this.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      this.data_slice = [
        //device.data.ms.slice(data_slice_from,data_slice_to),
        this.data["A"+0].slice(data_slice_from,data_slice_to),
        this.data["A"+1].slice(data_slice_from,data_slice_to),
        this.data["A"+2].slice(data_slice_from,data_slice_to),
        this.data["A"+3].slice(data_slice_from,data_slice_to),
        this.data["A"+4].slice(data_slice_from,data_slice_to),
        this.data["A"+5].slice(data_slice_from,data_slice_to),
        this.data["A"+6].slice(data_slice_from,data_slice_to),
        this.data["A"+7].slice(data_slice_from,data_slice_to),
        this.data["A"+8].slice(data_slice_from,data_slice_to),
        this.data["A"+9].slice(data_slice_from,data_slice_to),
        this.data["A"+10].slice(data_slice_from,data_slice_to),
        this.data["A"+11].slice(data_slice_from,data_slice_to),
        this.data["A"+12].slice(data_slice_from,data_slice_to),
        this.data["A"+13].slice(data_slice_from,data_slice_to),
        this.data["A"+14].slice(data_slice_from,data_slice_to),
        this.data["A"+15].slice(data_slice_from,data_slice_to),
        this.data["A"+16].slice(data_slice_from,data_slice_to),
        this.data["A"+17].slice(data_slice_from,data_slice_to),
        this.data["A"+18].slice(data_slice_from,data_slice_to),
        this.data["A"+19].slice(data_slice_from,data_slice_to),
        this.data["A"+20].slice(data_slice_from,data_slice_to),
        this.data["A"+21].slice(data_slice_from,data_slice_to),
        this.data["A"+22].slice(data_slice_from,data_slice_to),
        this.data["A"+23].slice(data_slice_from,data_slice_to),
        this.data["A"+24].slice(data_slice_from,data_slice_to),
        this.data["A"+25].slice(data_slice_from,data_slice_to),
        this.data["A"+26].slice(data_slice_from,data_slice_to),
        this.data["A"+27].slice(data_slice_from,data_slice_to),
        this.data["A"+28].slice(data_slice_from,data_slice_to),
        this.data["A"+29].slice(data_slice_from,data_slice_to),
        this.data["A"+30].slice(data_slice_from,data_slice_to),
        this.data["A"+31].slice(data_slice_from,data_slice_to)
        ];
        //this.lastnewLines=this.bufferednewLines;

    if(this.ready_to_send_data&&this.data_slice.length){
      this.ready_to_send_data = false;
(async () => {
  google.colab.kernel.comms.registerTarget('comms_testing', (comm, message) => {
    comm.send(JSON.stringify(this.data_slice), {buffers: message.buffers});
    this.data_slice=[];
    this.ready_to_send_data = true;
    //document.body.appendChild(document.createTextNode(JSON.stringify(message.buffers)))
    document.body.appendChild(document.createTextNode('comm opened.'))
  });
})()

    }
        }
}

device = new eeg32();
//
//

    connect = async () => {
        await this.device.setupSerialAsync();
    }

    disconnect = () => {
        if (this.ui) this.ui.deleteNode()
        this.device.closePort();
    }

//const photo = document.createElement('img');
      
//this.connect();
//return eeg.data;

        const audio = document.createElement('audio');

        const canvas = document.createElement('canvas');

          var ctx = canvas.getContext("2d");

      var image = new Image();
      image.onload = function() {
        ctx.drawImage(image, 0, 0);
      };
const div = document.createElement('div');
const btnconnect = document.createElement('button');
const btndisconnect = document.createElement('button');
const capture = document.createElement('button');
                  
    async function takePhoto2(quality=1) {
      btnconnect.remove();
      capture.remove();
      device.ready_to_send_data = true;
    }


    async function takePhoto(quality=1) {

      btnconnect.textContent = 'connect';
      div.appendChild(btnconnect);
      //capture1.onclick = 'navigator.serial.requestPort();';
      //capture1.onclick = function(event) {this.connect();};
      btnconnect.onclick = this.connect;
      //capture1.addEventListener("click", this.connect, false);

      btndisconnect.textContent = 'disconnect';
      div.appendChild(btndisconnect);
      //capture1.onclick = 'navigator.serial.requestPort();';
      //capture1.onclick = function(event) {this.connect();};
      btndisconnect.onclick = this.disconnect;
      //capture1.addEventListener("click", this.connect, false);

      //const div = document.createElement('div');
      capture.textContent = 'Capture';
      capture.onclick = takePhoto2;
      div.appendChild(capture);

      //const photo = document.createElement('img');
      //photo.textContent = 'photo';
      //div.appendChild(photo);

      div.appendChild(canvas);
      div.appendChild(audio);


///      const video = document.createElement('video');
///      video.style.display = 'block';
///      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
///      div.appendChild(video);
///      video.srcObject = stream;
///      await video.play();

      // Resize the output to fit the video element.
///      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);
      //if(quality==0.8)
      {
        //await new Promise((resolve) => connect.onclick = resolve);
        await new Promise((resolve) => capture.onclick = resolve);
      }
      

//this.connect();
    //device.port =  navigator.serial.requestPort();
		//navigator.serial.addEventListener("disconnect",(e) => {
		//	device.closePort(device.port);
		//});
		//device.onPortSelected(device.port,device.baudrate);

//var results = [];
//results.push(device.data.ms.slice(data_slice_from,data_slice_to);
//for(var i = 0; i < 32; i++){
//  var channel = "A"+i;
  //results.push(device.data[channel].slice(data_slice_from,data_slice_to);
//}

///      const canvas = document.createElement('canvas');
///      canvas.width = video.videoWidth;
///      canvas.height = video.videoHeight;
///      canvas.getContext('2d').drawImage(video, 0, 0);
///      stream.getVideoTracks()[0].stop();
///      div.remove();
      btnconnect.remove();
      capture.remove();
      device.ready_to_send_data = true;

      //return results;
      //return device.connected;
      //data_slice_from=0;
      ///data_slice_from=device.data.count-1-(512/8);
      ///data_slice_to=device.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      
    }




    async function takePhoto1(quality=1) {

///      const div = document.createElement('div');
///      const capture1 = document.createElement('button');
///      capture1.textContent = 'Capture1';
///      div.appendChild(capture1);
      //capture1.onclick = 'navigator.serial.requestPort();';
      //capture1.onclick = function(event) {this.connect();};
///      capture1.onclick = this.connect;
      //capture1.addEventListener("click", this.connect, false);

      //const div = document.createElement('div');
///      const capture = document.createElement('button');
///      capture.textContent = 'Capture';
///      div.appendChild(capture);

///      const video = document.createElement('video');
///      video.style.display = 'block';
///      const stream = await navigator.mediaDevices.getUserMedia({video: true});

///      document.body.appendChild(div);
///      div.appendChild(video);
///      video.srcObject = stream;
///      await video.play();

      // Resize the output to fit the video element.
///      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);
///      if(quality==0.8)
////      {
        //await new Promise((resolve) => capture.onclick = resolve);
///      }
      

//this.connect();
    //device.port =  navigator.serial.requestPort();
		//navigator.serial.addEventListener("disconnect",(e) => {
		//	device.closePort(device.port);
		//});
		//device.onPortSelected(device.port,device.baudrate);

//var results = [];
//results.push(device.data.ms.slice(data_slice_from,data_slice_to);
//for(var i = 0; i < 32; i++){
//  var channel = "A"+i;
  //results.push(device.data[channel].slice(data_slice_from,data_slice_to);
//}

///      const canvas = document.createElement('canvas');
///      canvas.width = video.videoWidth;
///      canvas.height = video.videoHeight;
///      canvas.getContext('2d').drawImage(video, 0, 0);
///      stream.getVideoTracks()[0].stop();
///      div.remove();
      
      var data_slice_send=this.device.data_slice;
      device.bufferednewLines=0;
      return data_slice_send;
      //return results;
      //return device.connected;
      ///data_slice_from=device.data.count-1-512*(5*1/8+0.1);
      ///data_slice_to=device.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      
    }


    async function displayPhoto1(photodata,photoWidth,photoHeight) {
      //const canvas = document.createElement('canvas');
      canvas.width = photoWidth;
      canvas.height = photoHeight;
      //canvas.getContext('2d').drawImage(photodata, 0, 0);
      //var canvas = document.getElementById("c");
      ///var ctx = canvas.getContext("2d");

      ///var image = new Image();
      ///image.onload = function() {
      ///  ctx.drawImage(image, 0, 0);
      ///};
      image.src = photodata;
    }
    async function displayPhoto2(photodata,photoWidth,photoHeight) {
      //const canvas = document.createElement('canvas');
      //canvas.width = photoWidth;
      //canvas.height = photoHeight;
      //canvas.getContext('2d').drawImage(photodata, 0, 0);
      //var canvas = document.getElementById("c");
      ///var ctx = canvas.getContext("2d");

      ///var image = new Image();
      ///image.onload = function() {
      ///  ctx.drawImage(image, 0, 0);
      ///};
      //image.src = photodata;
      const senderChannel = new BroadcastChannel("channel");
      senderChannel.postMessage(photodata);


      //photo.src = "data:image/png;base64," + photodata

///      const div = document.createElement('div');
///      const capture1 = document.createElement('button');
///      capture1.textContent = 'Capture1';
///      div.appendChild(capture1);
      //capture1.onclick = 'navigator.serial.requestPort();';
      //capture1.onclick = function(event) {this.connect();};
///      capture1.onclick = this.connect;
      //capture1.addEventListener("click", this.connect, false);

      //const div = document.createElement('div');
///      const capture = document.createElement('button');
///      capture.textContent = 'Capture';
///      div.appendChild(capture);

///      const video = document.createElement('video');
///      video.style.display = 'block';
///      const stream = await navigator.mediaDevices.getUserMedia({video: true});

///      document.body.appendChild(div);
///      div.appendChild(video);
///      video.srcObject = stream;
///      await video.play();

      // Resize the output to fit the video element.
///      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);
///      if(quality==0.8)
////      {
        //await new Promise((resolve) => capture.onclick = resolve);
///      }
      

//this.connect();
    //device.port =  navigator.serial.requestPort();
		//navigator.serial.addEventListener("disconnect",(e) => {
		//	device.closePort(device.port);
		//});
		//device.onPortSelected(device.port,device.baudrate);

//var results = [];
//results.push(device.data.ms.slice(data_slice_from,data_slice_to);
//for(var i = 0; i < 32; i++){
//  var channel = "A"+i;
  //results.push(device.data[channel].slice(data_slice_from,data_slice_to);
//}

///      const canvas = document.createElement('canvas');
///      canvas.width = video.videoWidth;
///      canvas.height = video.videoHeight;
///      canvas.getContext('2d').drawImage(video, 0, 0);
///      stream.getVideoTracks()[0].stop();
///      div.remove();


      //return results;
      //return device.connected;
      //data_slice_from=device.data.count-1-512;
      //data_slice_to=device.data.count-1;
      //return device.data.count;
      //data_length=5120;
      //return device.getLatestData();
      //return device.data;
      //return canvas.toDataURL('image/jpeg', quality);
      
    }
  //const senderChannel = new BroadcastChannel("channel");

/*(async () => {
const channel = await google.colab.kernel.comms.open('comm_target', '[[1,2],[3,4]]', []);
  let success = false;
  for await (const message of channel.messages) {
    if (message.data.response == 'got comm open!') {
      //document.body.appendChild(document.createTextNode(JSON.stringify([message])));
      //const responseBuffer = new Uint8Array(message.buffers[0]);
      //for (let i = 0; i < buffer.length; ++i) {
      //  if (responseBuffer[i] != buffer[i]) {
      //    console.error('comm buffer different at ' + i);
      //    return;
      //  }
      //}
      // Close the channel once the expected message is received. This should
      // cause the messages iterator to complete and for the for-await loop to
      // end.
      channel.close();
    }
  }

})()*/

    async function playAudio1(audiodata,photoWidth,photoHeight) {
      //const canvas = document.createElement('canvas');
      canvas.width = photoWidth;
      canvas.height = photoHeight;
      audio.controls = true;
      audio.autoplay = true;

      //canvas.getContext('2d').drawImage(photodata, 0, 0);
      //var canvas = document.getElementById("c");
      ///var ctx = canvas.getContext("2d");

      ///var image = new Image();
      ///image.onload = function() {
      ///  ctx.drawImage(image, 0, 0);
      ///};
      audio.src = audiodata;
      audio.play()
    }

    ''')
  display(js)  
  data = eval_js('takePhoto({})'.format(quality))
  #binary = b64decode(data.split(',')[1])
  #data = eval_js('device.connected')
  #data = eval_js('device.connected')
  #data = eval_js('capture.click()')
  #data = eval_js('navigator.serial.requestPort()')
  

  #		this.connected = false;
	#	this.subscribed = false;
  #      this.buffer = [];
  #     this.startByte = 160; // Start byte value
	#	this.stopByte = 192; // Stop byte value
	#	this.searchString = new Uint8Array([this.stopByte,this.startByte]); //Byte search string
	#	this.readRate = 16.666667; //Throttle EEG read speed. (1.953ms/sample min @103 bytes/line)
	#	this.readBufferSize = 2000; //Serial read buffer size, increase for slower read speeds (~1030bytes every 20ms) to keep up with the stream (or it will crash)

  #		this.sps = 512; // Sample rate
	#	this.nChannels = 32;

  #data = eval_js('device.getLatestData()')
  #print(data)
  #binary = b64decode(data)
  #with open(filename, 'wb') as f:
  #  f.write(binary)
  #return filename
  return data

In [7]:
stepSize = 1/pow(2,24)
vref = 2.50 #2.5V voltage ref +/- 250nV
gain = 8

vscale = (vref/gain)*stepSize #volts per step.
uVperStep = 1000000 * ((vref/gain)*stepSize) #uV per step.
scalar = 1/(1000000 / ((vref/gain)*stepSize)) #steps per uV.


In [8]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [9]:
print('Tensorflow version: {}'.format(tf.__version__) )

Tensorflow version: 1.15.2


In [10]:
%cd /content
#!git clone https://github.com/NVlabs/stylegan2.git
#!git clone https://github.com/NVlabs/stylegan2-ada.git
#!git clone https://githgitub.com/cyrilzakka/stylegan2-tpu.git
#!wget https://github.com/shawwn/stylegan2/archive/refs/heads/working.zip
#!unzip working.zip
#%ls

/content


In [11]:
#%cd /content/stylegan2-ada
#%cd /content/stylegan2-tpu
#%cd stylegan2-working

In [12]:
!pip install googledrivedownloader

In [13]:
from google_drive_downloader import GoogleDriveDownloader as gdd
files_path = [['1ZJir-_ls92s56LFmw_HVuQ7ANqFFN5WG', '/content/model/model.ckpt-18637.data-00000-of-00001'],
              ['1d5ayi4w-70AvKYPk-8sXzsSzpK1jMRgm', '/content/model/model.ckpt-18637.index'],  
              ['15CWn0yK3FKsHbAOGNLQYVg4eZC1oNIrL', '/content/model/model.ckpt-18637.meta'],  
              ['1x5QEFeoochk-rhvtvJc98kIB5_SAwn0u', '/content/model/args.txt'],  
              ['1UgSZaBTCTDXaPbfv8l0wmpHbD5u051o5', '/content/model/graph.pbtxt'],  
              ['1LGfAkuOFvA3NdFE_rOq9WyeXGGgEOf0F', '/content/model/checkpoint'],  
              ['1bPD0bXCC_18oWbUjmjkacF-CShlA6yNd', '/content/model/infer/infer.pbtxt'],  
              ['13OQuRx7Ku6KJ9o9FU-JN3yB0Njul9Vem', '/content/model/infer/infer.meta']]
for i in range(len(files_path)):
  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])


In [14]:
# Load the model
tf.reset_default_graph()
saver = tf.train.import_meta_graph('/content/model/infer/infer.meta')
graph = tf.get_default_graph()
sess = tf.InteractiveSession()
saver.restore(sess, f'/content/model/model.ckpt-18637')
dim = 100
break_len = 65536

z = graph.get_tensor_by_name('z:0')
G_z = graph.get_tensor_by_name('G_z:0')# Load the model
tf.reset_default_graph()
saver = tf.train.import_meta_graph('/content/model/infer/infer.meta')
graph = tf.get_default_graph()
sess = tf.InteractiveSession()
saver.restore(sess, f'/content/model/model.ckpt-18637')
dim = 100
break_len = 65536

z = graph.get_tensor_by_name('z:0')
G_z = graph.get_tensor_by_name('G_z:0')

INFO:tensorflow:Restoring parameters from /content/model/model.ckpt-18637
INFO:tensorflow:Restoring parameters from /content/model/model.ckpt-18637


/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [15]:
import numpy as np
from IPython.display import display, Audio
#from google.colab import files
import scipy.io.wavfile
import matplotlib.pyplot as plt
%matplotlib inline
!mkdir "./neuralfunk examples"

def generate_trajectory(n_iter, _z0=None, mov_last=None, jump=0.3, smooth=0.3, include_z0=True):
    _z = np.empty((n_iter + int(not include_z0), dim))
    _z[0] = _z0 if _z0 is not None else np.random.random(dim)*2-1
    mov = mov_last if mov_last is not None else (np.random.random(dim)*2-1)*jump
    for i in range(1, len(_z)):
        mov = mov*smooth + (np.random.random(dim)*2-1)*jump*(1-smooth)
        mov -= (np.abs(_z[i-1] + mov) > 1) * 2 * mov
        _z[i] = _z[i-1] + mov
    return _z[-n_iter:], mov
    
import numpy as np
from IPython.display import display, Audio
#from google.colab import files
import scipy.io.wavfile
import matplotlib.pyplot as plt
%matplotlib inline
!mkdir "./neuralfunk examples"

def generate_trajectory(n_iter, _z0=None, mov_last=None, jump=0.3, smooth=0.3, include_z0=True):
    _z = np.empty((n_iter + int(not include_z0), dim))
    _z[0] = _z0 if _z0 is not None else np.random.random(dim)*2-1
    mov = mov_last if mov_last is not None else (np.random.random(dim)*2-1)*jump
    for i in range(1, len(_z)):
        mov = mov*smooth + (np.random.random(dim)*2-1)*jump*(1-smooth)
        mov -= (np.abs(_z[i-1] + mov) > 1) * 2 * mov
        _z[i] = _z[i-1] + mov
    return _z[-n_iter:], mov

mkdir: cannot create directory ‘./neuralfunk examples’: File exists


In [18]:
#import pretrained_networks
from google_drive_downloader import GoogleDriveDownloader as gdd

#url = 'https://drive.google.com/open?id=1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez'
#anime2:
#url = 'https://drive.google.com/open?id=1YckI8gwqPbZBI8X4eaQAJCgWx-CqCTdi'
#anime:
#url = 'https://drive.google.com/open?id=1aUrChOhq5jDEddZK1v_Dp1vYNlHSBL9o'
#abstract:
url = 'https://drive.google.com/open?id=1ie1vWw1JNsfrZWRtMvhteqzVz4mt4KGa'
#'https://drive.google.com/open?id=1BHeqOZ58WZ-vACR2MJkh1ZVbJK2B-Kle'
model_id = url.replace('https://drive.google.com/open?id=', '')

network_pkl = '/content/models/model_%s.pkl' % model_id#(hashlib.md5(model_id.encode()).hexdigest())
##gdd.download_file_from_google_drive(file_id=model_id, dest_path=network_pkl)
#_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

##import pickle

##import dnnlib
##import dnnlib.tflib as tflib
#
#tflib.init_tf()

#with dnnlib.util.open_url(network_pkl) as fp:
#        _G, _D, Gs = pickle.load(fp)


In [17]:
##import dnnlib 
##import dnnlib.tflib as tflib 
import PIL.Image 
from tqdm import tqdm

import dnnlib
import dnnlib.tflib as tflib
import PIL.Image
from tqdm import tqdm

noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

def generate_images(zs, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    for z_idx, z in tqdm(enumerate(zs)):
        Gs_kwargs.truncation_psi = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
    return imgs

def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

In [19]:
import numpy as np

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

In [20]:
!pip install tqdm

In [21]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [22]:
#!mkdir /content/eeg
#!cp -r -v "/content/gdrive/MyDrive/EEG-GAN-audio-video/eeg/record-[2019.11.13-22.23.59].csv" "/content/eeg/"
#!cp -r -v "/content/gdrive/MyDrive/EEG-GAN-audio-video/eeg/record-[2020.06.28-14.26.09].csv" "/content/eeg/"

In [23]:
#from google_drive_downloader import GoogleDriveDownloader as gdd
!mkdir /content/eeg

#csv_name = 'record-[2020.06.28-14.26.09].csv'
#url = 'https://drive.google.com/open?id=1LtMfr9GduR3semMVgh_6JoUSKbCG8XbH'
#csv_name = 'record-[2019.11.13-22.23.59].csv'
#url = 'https://drive.google.com/open?id=1Nfzi6yT83SBZxtgIYVtYp7C7g_Sq9OdS'
#csv_id = url.replace('https://drive.google.com/open?id=', '')
#csv_path = '/content/eeg/%s' % csv_name
#gdd.download_file_from_google_drive(file_id=csv_id,
#                                    dest_path=csv_path)

#csv_name = 'record-[2019.11.13-22.23.59].csv'
#url = 'https://drive.google.com/open?id=1Nfzi6yT83SBZxtgIYVtYp7C7g_Sq9OdS'
#csv_id = url.replace('https://drive.google.com/open?id=', '')
#csv_path = '/content/eeg/%s' % csv_name
#gdd.download_file_from_google_drive(file_id=csv_id,
#                                    dest_path=csv_path)

In [24]:
!pip install mne


     |████████████████████████████████| 6.9 MB 6.7 MB/s 


In [25]:
import mne
from mne import io
from mne.datasets import sample
from mne.minimum_norm import read_inverse_operator, compute_source_psd

from mne.connectivity import spectral_connectivity, seed_target_indices

import pandas as pd
import numpy as np


In [26]:
    key = 0
    idx = 0

    data_path = '/content/eeg'
    #raw_fname = data_path + '/record-[2019.11.13-22.23.59].gdf'
    #raw = mne.io.read_raw_gdf(raw_fname, preload=True)

    path = data_path

    #ch_names = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','FP2','Fz','Cz']
    #data = pd.read_csv(path + '/record-[2019.11.13-22.23.59].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    
    #ch_names = ['FP1','F3','P3','O1','O2','P4','F4','FP2']
    #data = pd.read_csv(path + '/record-[2020.06.28-14.26.09].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    
    #data = pd.read_csv(path + '/record-[2020.06.28-00.36.11].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    #data = pd.read_csv(path + '/record-[2020.06.29-19.49.23].csv', skiprows=0, usecols=ch_names, header=0, delimiter=';') 
    
    #print(data)
    #data_transpose=np.transpose(data)

    sfreq = 512 
    ch_types=['eeg']*len(ch_names)
    info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types=ch_types)
    #info = mne.create_info(sfreq = sfreq)
    #raw = mne.io.RawArray(data_transpose, info)
    ##raw = mne.io.RawArray(data_uv, info)
    #raw = mne.io.RawArray(data, info)
    #raw.plot()

    # Setup for reading the raw data
    #raw = io.read_raw_fif(raw_fname, verbose=False)
    #events = mne.find_events(raw, stim_channel='STI 014')
    #inverse_operator = read_inverse_operator(fname_inv)
    #raw.info['bads'] = ['MEG 2443', 'EEG 053']

    # picks MEG gradiometers
    #picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False)
    #picks = ch_names
    

In [27]:
#!pip install imageio==2.9
!pip install imageio==2.4.1
!pip install imageio-ffmpeg==0.4.3 pyspng==0.1.0

     |████████████████████████████████| 26.9 MB 89 kB/s 
     |████████████████████████████████| 195 kB 63.9 MB/s 


In [28]:
!mkdir '/content/out'

In [29]:
#for ch in ch_names:
#  raw.set_channel_types({ch:'eeg'})
#raw.crop(tmax=150)
#raw.crop(tmax=150).resample(100).pick('eeg')



In [30]:
bands = [[8.,12.]]
#bands = [[4.,7.],[8.,12.],[13.,29.]]
#bands = [[8.,12.],[8.,12.],[8.,12.]]
#bands_name = ['theta','alpha','beta']
##datas=[]
##for band in range(len(bands)):
## datas.append(raw)
# datas.append(raw.filter(l_freq=bands[band][0], h_freq=bands[band][1],method='iir'))
#theta_data = raw.filter(l_freq=4, h_freq=7,method='iir')
#alpha_data = raw.filter(l_freq=8, h_freq=12,method='iir')
#beta_data = raw.filter(l_freq=13, h_freq=29,method='iir')

In [31]:
methods = ['coh']
#methods = ['ciplv']
#methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'wpli']
##epochs = []
##duration=5*1/8
##overlap=5*1/8-0.1
##for band in range(len(bands)):display
# epochs.append(mne.make_fixed_length_epochs(datas[band], 
#                                            duration=0.1, preload=False))
## epochs.append(mne.make_fixed_length_epochs(datas[band], 
##                                            duration=5*1/8, preload=False, overlap=5*1/8-0.1))
# epochs.append(mne.make_fixed_length_epochs(datas[band], duration=1.25, preload=False, overlap=1.15))
#epochs = [mne.make_fixed_length_epochs(theta_data, duration=0.1, preload=False),
#          mne.make_fixed_length_epochs(alpha_data, duration=0.1, preload=False),
#          mne.make_fixed_length_epochs(beta_data, duration=0.1, preload=False)]
#epochs = mne.make_fixed_length_epochs(alpha_data, duration=0.1, preload=False)
#epochs = mne.make_fixed_length_epochs(raw, duration=2, preload=True)
#epochs = mne.make_fixed_length_epochs(raw, duration=10, preload=True)
#epochs = mne.make_fixed_length_epochs(raw, duration=1, preload=False)
#event_related_plot = epochs.plot_image(picks=['FP1'])


import matplotlib.pyplot as plt

corr_matrix = mne.connectivity.envelope_correlation(alpha_data, combine=None)

first_30 = corr_matrix[0]
last_30 = corr_matrix[-1]
corr_matrices = [first_30, last_30]
color_lims = np.percentile(np.array(corr_matrices), [5, 95])
titles = ['First 30 Seconds', 'Last 30 Seconds']

fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Correlation Matrices from First 30 Seconds and Last 30 Seconds')
for ci, corr_matrix in enumerate(corr_matrices):
    ax = axes[ci]
    mpbl = ax.imshow(corr_matrix, clim=color_lims)
    ax.set_xlabel(titles[ci])
fig.subplots_adjust(right=0.8)
cax = fig.add_axes([0.85, 0.2, 0.025, 0.6])
cbar = fig.colorbar(ax.images[0], cax=cax)
cbar.set_label('Correlation Coefficient')

In [32]:
#fmin=8.
#fmax=13.age/png;base64," + notypI96vqmV1sGr2MGp1r6l1Lyh0rmd0bedzrSZ0Lac1Lyh2MGk2MGj1r6e18Ch1r+g1r6f17+f17+e176e2MCg2cKi2cOj28el18Kg1cGf1MCe0r2Z0ryX072W07yV0buTzLWNyLGLyLGNybKOzriU1L6b1sCd0ryZzriWybKSw6uMvKODvKOBw6iEya+MxquHx6yGz7SN2r+Z4cii3sag176a07ybzbSUzLKTzLGRzrKR0riX1buZ176b2cCb172X1ryVz7SMzLCIzbKK1LmR1LmSz7SM0beRzrSNzbOMyKyFxamExqmEzbCJzrGKy66Hy6+Jya2GzK+HzbCHzrGI0LOL0bSL0rSKzK6ExqmCxqiCza+JzbCIzK+Jx6qFw6WBxKWAyKmDyaqExaZ/yKqEyqyFyqyFzK+IzK+Iyq2Fx6qCw6Z+yKuC0LOJ0raMzrGJyayEyqyEy66Fy6+Ix6qCwKJ6wKJ6x…uVu6uVva6ZuauWu66bu6+du7Ggwrqpxb6tw7yrv7mou7amuLOju7eluLSgtLGbtbGasq6Xurefw8CnvrqeuLKUwbqYxb2ZycGdzMSizMSkzsaoycGjv7aWvbOTwLWWu7CSvLCRvK+QvbCRv7KUvK+SvLCUvLGYw7ukysSuzcezzMi2zsu6y8e4ycW2yse3ysa2zsy7y8i3ysezxsKswr6pwr6qw8Ctx8Wxycawx8OtxMGtyse00M271dTC1NTF1tjN1dnS2NzV2t7X2t7Y19vVztPOv8S+v8O9zNDJ09fPztLIys3BzdDC0tTG0NLC0dK819e819W409CxzsqryMSm1dK22te92Na/29vH4ODP3dzM2NfF0M6+y8m6yca4yMS0zcm3zsq2zcm2zsq3zcq3zsy5ysm1ycey0My0z8mwxr6iyb+fybyax7qWybuXy72bz8GgzL+fyb2ey7+izMClyb2jx7uiv7SavbKavLGavbGZuq6Uva
fmin=bands[0][0]
fmax=bands[0][1]+1.
          
##sfreq = raw.info['sfreq']  # the sampling frequency
#con_methods = ['coh', 'pli', 'ciplv']
con_methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'wpli']
#con_methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli', 'wpli2_debiased']
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
#    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=10)
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs[1][2000:2010], method=con_methods, mode='multitaper', sfreq=sfreq, 
#    faverage=True, mt_adaptive=True, n_jobs=1)
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs[0][2000,2001], method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
#    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
#con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs[0][2000:2010], method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
#    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)ed
###con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
##    epochs[0][0:10], method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
##    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

In [33]:
#freqs1= [[4.,7.],[8.,13.],[14.,30.]]

In [34]:
#for freq, ii in freqs1:
# sfreq = raw.info['sfreq']  # the sampling frequency
# con_methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'wpli']
# #con_methods = ['coh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli', 'wpli2_debiased']
# con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
#    epochs, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=freq[0],
#    fmax=freq[1], faverage=True, mt_adaptive=True, n_jobs=10)


In [35]:
##con_res = dict()
##for method, c in zip(con_methods, con):
##    con_res[method] = c[:, :, 0]

In [36]:
from mne.viz import circular_layout, plot_connectivity_circle
import matplotlib.pyplot as plt

label_names = ch_names

#plot_connectivity_circle(con_res['pli'], label_names, 
#                                                  title='All-to-All Connectivity (PLI)')
#fig = plt.figure(num=None, figsize=(8, 4), facecolor='black')
no_names = [''] * len(label_names)
##for ii, method in enumerate(con_methods):
##    plot_connectivity_circle(con_res[method], label_names, n_lines=300,
##                             title=method)
    #plot_connectivity_circle(con_res[method], label_names, n_lines=300,
    #                         title=method, padding=0, fontsize_colorbar=6,
    #                         fig=fig, subplot=(1, 8, ii + 1))

    #plot_connectivity_circle(con_res[method], no_names, n_lines=300,
    #                         title=method, padding=0, fontsize_colorbar=6,
    #                         fig=fig, subplot=(1, 6, ii + 1))
##plt.show()

In [37]:
%%javascript
        const canvas = document.createElement('canvas');

          var ctx = canvas.getContext("2d");

      var image = new Image();
      image.onload = function() {
        ctx.drawImage(image, 0, 0);
      };

      const div = document.createElement('div');

      div.appendChild(canvas);


      document.body.appendChild(div);

    async function displayPhoto1(photodata,photoWidth=400,photoHeight=400) {
      //const canvas = document.createElement('canvas');
      canvas.width = photoWidth;
      canvas.height = photoHeight;
      //canvas.getContext('2d').drawImage(photodata, 0, 0);
      //var canvas = document.getElementById("c");
      ///var ctx = canvas.getContext("2d");

      ///var image = new Image();
      ///image.onload = function() {
      ///  ctx.drawImage(image, 0, 0);
      ///};
      image.src = photodata;
    }

const listenerChannel = new BroadcastChannel('channel');
listenerChannel.onmessage = (msg) => {
  displayPhoto1(msg.data);
  //image.src = msg.data;
};

<IPython.core.display.Javascript object>

In [38]:
%%javascript
const senderChannel = new BroadcastChannel('channel');
senderChannel.postMessage('data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg=='
);

<IPython.core.display.Javascript object>

In [39]:
#myimage = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x05\x00\x00\x00\x05\x08\x06\x00\x00\x00\x8do&\xe5\x00\x00\x00\x1cIDAT\x08\xd7c\xf8\xff\xff?\xc3\x7f\x06 \x05\xc3 \x12\x84\xd01\xf1\x82X\xcd\x04\x00\x0e\xf55\xcb\xd1\x8e\x0e\x1f\x00\x00\x00\x00IEND\xaeB`\x82'
#encoded= "data:image/png;base64,"+base64.b64encode(myimage).decode()
#print(encoded)



In [40]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import base64

js = Javascript('''
  const senderChannel = new BroadcastChannel("channel");
    ''')
display(js)
#image_js="data:image/png;base64,b'iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg=='"

myimage = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x05\x00\x00\x00\x05\x08\x06\x00\x00\x00\x8do&\xe5\x00\x00\x00\x1cIDAT\x08\xd7c\xf8\xff\xff?\xc3\x7f\x06 \x05\xc3 \x12\x84\xd01\xf1\x82X\xcd\x04\x00\x0e\xf55\xcb\xd1\x8e\x0e\x1f\x00\x00\x00\x00IEND\xaeB`\x82'


encoded= "data:image/png;base64,"+base64.b64encode(myimage).decode()

#eval_js('senderChannel.postMessage({});'.format(image_js))
#eval_js('senderChannel.postMessage({})'.format(encoded))


<IPython.core.display.Javascript object>

In [57]:

_G=None
_D=None 
Gs=None

##tf_init=False
##if True:
##  if not tf_init:
##    tflib.init_tf()
##    with dnnlib.util.open_url(network_pkl) as fp:
##        _G, _D, Gs = pickle.load(fp)
##    tf_init = True


from IPython.display import Image
from pprint import pprint
import matplotlib.figure
import base64
from io import BytesIO

data_read=None
data_buffer=None
data_analyse=None

data_read = None

ready_to_get_data = True

from IPython.display import Javascript
import json
def target_func(comm, msg):
  # Only send the response if it's the data we are expecting.
  array_to_receive_as_json = msg['content']['data']
  global data_read
  data_read = json.loads(array_to_receive_as_json)
  print('got data')
  global ready_to_get_data
  ready_to_get_data = True
  if True:
  #if msg['content']['data'] == 'the data':
    comm.send({
          'response': 'got comm open!',
        }, None, msg['buffers']);
#print('data read init')
#get_ipython().kernel.comm_manager.register_target('comm_target', target_func)
#print('data read started')
ready_to_get_data = False

try:
  data = take_photo()
  #pprint(data)
  #print(data)
  #print(data[0][len(data[0])-1])
  
  # Show the image which was just taken.
  #display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))
device_connected=True
#print('back to data read')

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

#js = Javascript('''
#  const senderChannel = new BroadcastChannel("channel");
#    ''')
#display(js)
  #//const div = document.createElement('div');
  #//div.textContent = msg.data;
  #//document.body.appendChild(div);

import imageio
fps=10
out = imageio.get_writer('/content/out/output.mp4', mode='I', fps=fps, codec='libx264', bitrate='16M')
#for img in imgs:
#  out.append_data(np.asarray(img))
#out.close()

#print(eval_js('device.connected'))
#plt.ion()
#fig, axs = plt.subplots(6, 6)
#fig.tight_layout()
#fig = []
#for ii, method in enumerate(con_methods):
#  fig.append(matplotlib.figure.Figure());

js_image = ""
time000 = 0
time001 = 0
time002 = 0
time003 = 0
time004 = 0
time005 = 0
time006 = 0
time110 = 0
time100 = 0

##Gs_kwargs = dnnlib.EasyDict()
#Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False)
#t_nhwc = tf.transpose(t_nchw, axis=(0, 2, 3, 1))
##Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
##Gs_kwargs.randomize_noise = False
from time import perf_counter
time100=perf_counter()
import types

#await = lambda x: asyncio.get_event_loop().run_until_complete(x)

#loop = asyncio.get_event_loop()
#loop.create_task(some_async_function())

#data=None

from ipykernel import comm
buffer = b'hello world'
#print(buffer)
array_to_send = [[1,2],[3,4]]
array_to_send_as_json=json.dumps(array_to_send)
#channel = comm.Comm(target_name='comms_testing', data=array_to_send_as_json, buffers=[])

##channel = comm.Comm(target_name='comms_testing', data=array_to_send_as_json, buffers=[buffer])

message = None
array_to_receive_as_json=None
array_to_receive=None
def handle_message(msg):
  print('got data')
  #global message
  global array_to_receive_as_json
  global array_to_receive
  #message = msg
  print(msg)
  #print(msg['content']['data'])
  #print(message)
  #msg_parsed=json.loads(msg)
  #print(msg_parsed)
  array_to_receive_as_json=msg['content']['data']
  #print(array_to_receive_as_json)
  array_to_receive = json.loads(array_to_receive_as_json)
  #print(array_to_receive)
  global data_read
  data_read = array_to_receive
  global ready_to_get_data
  ready_to_get_data = True

##data_read = eval_js('takePhoto1({})'.format(0.8))
#print(type(data))
  
while (data_read is None):
  #if ready_to_get_data:
  #  ready_to_get_data = False
  #  channel = comm.Comm(target_name='comms_testing', data=array_to_send_as_json, buffers=[buffer])
  #  channel.on_msg(handle_message)
  #  print(channel)
    #print('will wait')
    #get_ipython().kernel.comm_manager.register_target('comm_target', target_func)
    #print('waiting')
  #data_read=data_read
  data_read = eval_js('takePhoto1({})'.format(0.8))
#print(type(data))
data_buffer=data_read
data_analyse=None
  #global js_image, data, analyse_ready, outimage_ready
  #analyse_ready=True
  #outimage_ready=True
                            
#import threading
#maxthreads = 3
#sema = threading.Semaphore(value=maxthreads)
#threads = list()
#threads.append(Thread(target=read_data, args=[]))
#threads.append(Thread(target=analyse, args=[]))
#threads.append(Thread(target=outimage, args=[]))
#threads[0].start()
#threads[1].start()
#threads[2].start()
#threads[0].join()
#threads[1].join()
#threads[2].join()

#while eval_js('device.connected'):
#while device_connected:
#  read_data()
#  analyse()
#  outimage()
  #device_connected=True
_G_z_full2 = None
while eval_js('device.connected'):
#while device_connected:
#def read_data():
  #global device_connected
  #while eval_js('device.connected'):
    #device_connected=True
  #  while device_connected:
    #global data_read, data_buffer, data_analyse
    time000=perf_counter()
    while (data_read is None):
      data_read = eval_js('takePhoto1({})'.format(0.8))
      #get_ipython().kernel.comm_manager.register_target('comm_target', target_func)
      #if ready_to_get_data:
      #  ready_to_get_data = False
      #  channel = comm.Comm(target_name='comms_testing', data=array_to_send_as_json, buffers=[buffer])
      #  channel.on_msg(handle_message)
      #data_read=data_read
    #print(type(data_read))
  
    if not (data_read is None):
      #print('read_data-data_read')
      #print(len(data_read[0]))
      data_buffer=np.concatenate((data_buffer,data_read),axis=1)
      data_read = None

      data_buffer_len=len(data_buffer[0])
      #print(data_buffer_len)
      data_slice_len=int(duration*512)
      #print(data_slice_len)
      if data_buffer_len>data_slice_len:
        data_buffer=data_buffer[0:len(data_buffer),data_buffer_len-data_slice_len:data_buffer_len]
      data_analyse=data_buffer
      #data_buffer=data_analyse
      #print('read_data-data_analyse')
      #print(len(data_analyse[0]))
  #device_connected=False
#async 
#def analyse():
  #global _G, _D, Gs, Gs_kwargs, network_pkl
  #global tf_init
  
  #if not tf_init:
  #  tflib.init_tf()
  #  with dnnlib.util.open_url(network_pkl) as fp:
  #      _G, _D, Gs = pickle.load(fp)
  #  tf_init = True
                
  #global device_connected
  #while device_connected:
    #print('analyse')
    #global data_analyse
    if not(data_analyse is None):
      data = data_analyse
      data_analyse=None
      #global sfreq,ch_types, info, bands, duration, overlap, ch_locations, methods, label_names, ch_names, uVperStep, fps, out
      #print('analyse-data')
      #print(len(data))
      time001=perf_counter()
      #print(data[0][len(data[0])-1])
      data_uv = [0]*len(label_names)
      for j in range(len(label_names)):
        data_uv[j]=[0]*len(data[ch_locations[j]])
        for i in range(len(data[ch_locations[j]])):
          data_uv[j][i] = data[ch_locations[j]][i] * uVperStep * 2
        #print(data_uv[j][0])
      #print(data_uv[0][0])

  
      time002=perf_counter()
      raw = mne.io.RawArray(data_uv, info, verbose=50)
      time003=perf_counter()
      datas=[]
      for band in range(len(bands)):
        datas.append(raw)
      time004=perf_counter()
      epochs = []
      for band in range(len(bands)):
        #epochs.append(mne.make_fixed_length_epochs(datas[band], duration=5*1/8, preload=False, overlap=5*1/8-0.1, verbose=50))
        epochs.append(mne.make_fixed_length_epochs(datas[band], duration=duration, preload=True, overlap=overlap, verbose=50))
        
      time005=perf_counter()
      #sfreq = raw.info['sfreq']  # the sampling frequency
      #con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
      #  epochs[0][0:10], method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
      #  fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1, verbose=50)
      #con_res = dict()
      #for method, c in zip(con_methods, con):
      #  con_res[method] = c[:, :, 0]
      #for ii, method in enumerate(con_methods):
      #  plot_connectivity_circle(con_res[method], label_names, n_lines=300, title=method, fig=fig, subplot=(6,6,1+ii))
      #plt.show()
      n_generate=int((float(len(data_uv[0]))/float(sfreq))/duration)
      #n_generate=1
      part_len = 10
    
      n_parts = n_generate//part_len
      if n_generate%part_len>0:
        n_parts=n_parts+1
    
      vol=1
      #vol=0.1
    
      psd_array=np.random.rand(part_len, dim) 
      time006=perf_counter()
    
      #Gs_kwargs = dnnlib.EasyDict()
      #Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
      #Gs_kwargs.randomize_noise = False
    
      imgs = []
      imgs1 = []
    
    
    #z_avg_samples=n_generate
    #for i in range(n_generate): # display separate audio for each break
      for j in range(n_parts): # display separate audio for each break
        for i in range(part_len): # display separate audio for each break
            ji = j * part_len + i
            
            #ji=n_generate-1
    
            if (i==0) and (n_generate-ji<part_len):
                psd_array=np.random.rand((n_generate-ji), dim) 
    
            eeg_step=ji
            #print (f'EEG step: {(eeg_step/3):.1f} s')
            tmin, tmax = 0+(eeg_step/fps), 2+(eeg_step/fps)  # use the first 120s of data
            #tmin, tmax = 0+(10*eeg_step/512), 2+(10*eeg_step/512)  # use the first 120s of data
            fmin, fmax = 0.5, 256  # look at frequencies between 4 and 100Hz
            #fmin, fmax = 0.5, 50  # look at frequencies between 4 and 100Hz
            #fmin, fmax = 8, 12  # look at frequencies between 4 and 100Hz
            #n_fft = 512  # the FFT size (n_fft). Ideally a power of 2
            n_fft = 1024  # the FFT size (n_fft). Ideally a power of 2
            #n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2
            #label = mne.read_label(fname_label)
            
            #print(str(ji) + '/' + str(n_generate))
            #logger = logging.getLogger()
            #logger.disabled = True
    
            sfreq = raw.info['sfreq']  # the sampling frequency
            
            psds=np.zeros(dim)
            
            for method in range(len(methods)):
             for band in range(len(bands)):
              #fmin=8.
              #fmax=13.
              fmin=bands[band][0]
              fmax=bands[band][1]
              #times[ji][0]=perf_counter()
              if band == 0:
               con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
                epochs[band][ji:ji+1], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
                fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=10, verbose=50)
              #times[ji][1]=perf_counter()
              # con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
              #  epochs[band][ji:ji+10], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
              #  fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1, verbose=50)
              #con, freqs, times, n_epochs, n_tapers = spectral_connectivity(
              #  epochs[band][ji,ji+1], method=methods[method], mode='multitaper', sfreq=sfreq, fmin=fmin,
              #  fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)
              psds_shift1=int(round(method*len(bands)+band)*(len(ch_names)*(len(ch_names)-1)/2))
              ji1=0
              for j1 in range(0,len(ch_names)): # display separate audio for each break
                for i1 in range(0,j1): # display separate audio for each break
                  psds[ji1+psds_shift1]=(con[j1][i1][0]-0.5)*1
                  ji1 = ji1+1
            #psds, freqs = mne.time_frequency.psd_welch(raw, picks=picks,
            #                 tmin=tmin, tmax=tmax, fmin=fmin, fmax=fmax,
            #                 n_fft=n_fft)
            #logger.disabled = False
    
            #print(freqs)
            #print(psds)
    
            #psd_array.append(psds)   
            #psd_array=np.random.rand(1, dim) 
            #psd_array[0]=psds
            psd_array[i]=psds
            #psd_array=[psd_array[len(psd_array)-1]]
            ##z_samples = psds
    
            #w_samples = G.mapping(torch.from_numpy(z_samples).to(device), None)  # [N, L, C]
            #w_samples = w_samples[:, :1, :].cpu().numpy().astype(np.float32)       # [N, 1, C]
            ##z_avg = np.mean(z_samples, axis=0)      # [1, 1, C]
            #z_avg = np.mean(z_samples, axis=0, keepdims=True)      # [1, 1, C]
            ##psd_array[i]=z_avg
            #psd_array[i]=z_avg
            #print(z_avg)
            #z_std = (np.sum((z_samples - z_avg) ** 2) / z_avg_samples) ** 0.5
    
            #psd_array[i]=psds
            #psds_transpose=np.transpose(psds)
            #plt.plot(freqs,psds_transpose)
            #plt.xlabel('Frequency (Hz)')
            #plt.ylabel('PSD (dB)')
            #plt.title('Power Spectrum (PSD)')
            #plt.show()
            if (i==part_len-1) or (ji==n_generate-1) :
                
                _z = psd_array * vol
                _G_z = sess.run(G_z, {z: _z})[:,:,0]
                if j==0:
                  _G_z_full=_G_z
                else:
                  _G_z_full=np.append(_G_z_full,_G_z)
                
                if _G_z_full2 is None:
                  _G_z_full2=_G_z_full
                else:
                  _G_z_full2=np.append(_G_z_full2,_G_z_full)

                ##_z = psd_array * vol
                #times[ji][2]=perf_counter()
                #print("run>")
                images=[]
                ##images = Gs.run(_z, None, **Gs_kwargs) # [minibatch, height, width, channel]
                #print("run<")
                #times[ji][3]=perf_counter()
                #images = Gs.run(_z, None, **Gs_kwargs) # [minibatch, height, width, channel]
                #images=[images[len(images)-1]]
                for image in images:
                  #imgs1.append(image)
                  image_pil=PIL.Image.fromarray(image, 'RGB')
                  image_asarray=np.asarray(image_pil)
                  out.append_data(image_asarray)
                  #image_pil = PIL.Image.fromarray(image, 'RGB')                  #Crée une image à partir de la matrice
                  #global xsize, ysize
                  #xsize=400
                  #ysize=400
                  img=image_pil.resize((xsize,ysize),PIL.Image.ANTIALIAS)
                  buffer = BytesIO()
                  img.save(buffer,format="JPEG")                  #Enregistre l'image dans le buffer
                  #img.save(buffer,format="PNG")                  #Enregistre l'image dans le buffer
                  buffer.seek(0)
                  myimage = buffer.getvalue()   
                  #display(image_pil) 
                  #display(img)                  
                  encoded= "data:image/png;base64,"+base64.b64encode(myimage).decode()
    
                  #encoded = base64.b64encode(image_asarray).decode('ascii')
                  #js='''this.photo.src = "data:image/png;base64,{0}"'''.format(encoded)

                  
                  #js='''senderChannel.postMessage("{0}")'''.format(encoded)
                  js='''displayPhoto1("{0}",{1},{2})'''.format(encoded,xsize,ysize)
                  #js='''displayPhoto2("{0}",{1},{2})'''.format(encoded,xsize,ysize)
                  #pprint(js)
                  #with open('/content/out/debug.txt', 'w') as f:
                  #  f.write(js)
                  ##eval_js(js)
                  #eval_js('displayPhoto1({})'.format())
                  #imgs.append(PIL.Image.fromarray(image, 'RGB'))
                #out.append(PIL.Image.fromarray(images[0], 'RGB'))
                #_G_z = sess.run(G_z, {z: _z})[:,:,0]
                #if j==0:
                #    _G_z_full=_G_z
                #else:
                #    _G_z_full=np.append(_G_z_full,_G_z)
                if (ji==n_generate-1) :
                    time110=perf_counter()
                    #eval_js(js)
                    #global js_image
                    ##js_image=js
                    #filename = f"./neuralfunk examples/neuralfunkv2_gen_fps{fps:.2f}_{120*2*fps:.2f}bpm_n{n_generate}_vol{vol:.2f}.wav"
                    #scipy.io.wavfile.write(filename, hz, _G_z.flatten()) # change rate for different tempo
                    buffer = BytesIO()
                    scipy.io.wavfile.write(buffer, hz, _G_z_full.flatten()) # change rate for different tempo
                    #wave.open(buffer, mode='wb')

                    buffer.seek(0)
                    mysound = buffer.getvalue()   
                    encoded= "data:audio/wav;base64,"+base64.b64encode(mysound).decode()
    
                  #encoded = base64.b64encode(image_asarray).decode('ascii')
                  #js='''this.photo.src = "data:image/png;base64,{0}"'''.format(encoded)

                  
                  #js='''senderChannel.postMessage("{0}")'''.format(encoded)
                    js='''playAudio1("{0}",{1},{2})'''.format(encoded,xsize,ysize)
                    js_image=js
                    #print('analyse-js_image')
                    #print(len(js_image)) 
                    #loop = asyncio.get_event_loop()
                    #loop.create_task(outimage(js))
    
                    #await outimage(js)
                    #await eval_js(js)
                    #await asyncio.get_event_loop().run_until_complete(eval_js(js))

                 
                    break
      #analyse_ready = True
#def outimage():
  #global device_connected
  #while device_connected:

   #await 
   #print('js')
   #global js_image
    if not(js_image==""):
      js=js_image
      js_image=""
      eval_js(js)
     #print('outimage-js')
      #print(len(js))
      if True:
#      if False:
                    time111=perf_counter()
                    ##print (f'000: {(time111-time000):.1f}s')
                    ##print (f'001: {(time111-time001):.1f}s')
                    #print (f'002: {(time111-time002):.1f} s')
                    #print (f'003: {(time111-time003):.1f} s')
                    #print (f'004: {(time111-time004):.1f} s')
                    #print (f'005: {(time111-time005):.1f} s')
                    #print (f'006: {(time111-time006):.1f} s')
                    ##print (f'110: {(time111-time110):.1f}s')
                    #print (f'111: {(time111-time111):.1f} s')
                    ##print (f'001-000: {(time001-time000):.1f}s, {(time001-time000)*100/(time111-time000):.0f}%')
                    ##print (f'110-001: {(time110-time001):.1f}s, {(time110-time001)*100/(time111-time000):.0f}%')
                    ##print (f'111-110: {(time111-time110):.1f}s, {(time111-time110)*100/(time111-time000):.0f}%')
                    
                    print (f'000: {(time111-time000):.1f}s, '+
                      f'001: {(time111-time001):.1f}s, '+
                      f'110: {(time111-time110):.1f}s, '+
                      f'001-000: {(time001-time000):.1f}s, {(time001-time000)*100/(time111-time000):.0f}%, '+
                      f'110-001: {(time110-time001):.1f}s, {(time110-time001)*100/(time111-time000):.0f}%. '+
                      f'111-110: {(time111-time110):.1f}s, {(time111-time110)*100/(time111-time000):.0f}%')
   #outimage_ready=True
  #device_connected=True

#loop = asyncio.get_event_loop()
#tasks = []
#tasks.append(asyncio.ensure_future(read_data()))
#tasks.append(asyncio.ensure_future(analyse()))
#tasks.append(asyncio.ensure_future(outimage()))
#loop.run_until_complete(asyncio.gather(*tasks))
#loop.close()
  
#async def main(): 
  #loop = asyncio.get_event_loop()
  #tasks = []
    
  #tasks.append(asyncio.create_task(read_data()))
  #tasks.append(asyncio.create_task(analyse()))
  #tasks.append(asyncio.create_task(outimage()))
  #tasks.append(asyncio.ensure_future(read_data()))
  #tasks.append(asyncio.ensure_future(analyse()))
  #tasks.append(asyncio.ensure_future(outimage()))
  #loop.run_until_complete(asyncio.gather(*tasks))
  #loop.close()
  #await tasks[0]
  #await tasks[1]
  #await tasks[2]
  
  #loop.run_until_complete(asyncio.wait(tasks, return_when=asyncio.ALL_COMPLETED))
  #loop.close()
#asyncio.run(main())

#while eval_js('device.connected'):
#  device_connected=True
#device_connected=False

#print(eval_js('device.connected'))

out.close()
from IPython import display as ipythondisplay
import io
import os
import base64
from IPython.display import HTML

def show_video(vid):
  #mp4list = [video
  #if len(mp4list) > 0:
  ext = os.path.splitext(vid)[-1][1:]
  video = io.open(vid, 'r+b').read()
  #encoded = base64.b64encode(video)
  ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
              loop controls style="height: 400px;">
              <source src="data:video/{1}';base64,{0}" type="video/{1}" />
              </video>'''.format(base64.b64encode(video).decode('ascii'), ext)))

scipy.io.wavfile.write('/content/out/output.wav', hz, _G_z_full2.flatten())

#show_video('/content/out/output.mp4')

<IPython.core.display.Javascript object>

000: 1.1s, 001: 1.1s, 110: 1.1s, 001-000: 0.0s, 0%, 110-001: 0.1s, 5%. 111-110: 1.1s, 95%
000: 3.2s, 001: 1.9s, 110: 1.8s, 001-000: 1.3s, 42%, 110-001: 0.1s, 2%. 111-110: 1.8s, 57%
000: 2.7s, 001: 1.2s, 110: 1.1s, 001-000: 1.5s, 56%, 110-001: 0.1s, 2%. 111-110: 1.1s, 42%
000: 2.3s, 001: 1.2s, 110: 1.2s, 001-000: 1.1s, 46%, 110-001: 0.1s, 3%. 111-110: 1.2s, 51%
000: 2.7s, 001: 1.2s, 110: 1.2s, 001-000: 1.5s, 54%, 110-001: 0.1s, 2%. 111-110: 1.2s, 43%
000: 5.2s, 001: 1.5s, 110: 1.5s, 001-000: 3.6s, 71%, 110-001: 0.1s, 1%. 111-110: 1.5s, 28%
000: 2.3s, 001: 1.4s, 110: 1.3s, 001-000: 0.9s, 39%, 110-001: 0.1s, 3%. 111-110: 1.3s, 58%


In [49]:
#!pip install playsound
#!pip install pygobject

In [ ]:
#from playsound import playsound
#playsound('/content/out/output.wav')

In [58]:
from IPython.display import Audio
Audio('/content/out/output.wav', autoplay=True)


In [ ]:
#print(eval_js('device.connected'))


In [ ]:
#stop

In [ ]:
#con_res['coh']

In [ ]:
#from mne.viz import plot_sensors_connectivity
#plot_sensors_connectivity(epochs.info, con[:, :, 0])

In [ ]:
#from mne.time_frequency import AverageTFR
#tfr = AverageTFR(epochs.info, con, times, freqs, len(epochs))
#tfr.plot_topo(fig_facecolor='w', font_color='k', border='k')

In [ ]:
#import logging
#logging.basicConfig(level='warning')


In [ ]:
#con

In [59]:
from google.colab import files
#files.download('/content/out/output.mp4')
files.download('/content/out/output.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#stop

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!cp -r -v "/content/out" "/content/gdrive/MyDrive/EEG-GAN-audio-video"

In [ ]:
stop

In [ ]:
imageio.plugins.ffmpeg.download()


In [ ]:
import moviepy.editor
duration_sec=n_generate/fps

def make_frame(t):
    frame_idx = int(np.clip(np.round(t * fps), 0, n_generate - 1))
    #latents = all_latents[frame_idx]
    return imgs1[frame_idx]

video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)

# Use this if you want to generate .mp4 video instead
# video_clip.write_videofile("interpolate3x2-2m.mp4", fps=fps, codec=codec, bitrate=bitrate)

video_clip.write_gif('/content/out/output.gif',fps=fps)



In [ ]:
!cp -r -v "/content/out" "/content/gdrive/MyDrive/EEG-GAN-audio-video"
